# Train YOLOv1

In [1]:
import shutil
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import sys
sys.path.append('../')
from torchsummary.torchsummary import summary
from dataloader import VOC
import numpy as np
import matplotlib.pyplot as plt
import yolov1
from yolov1 import detection_collate
from yolov1 import detection_loss
from yolov1 import save_checkpoint

# 0. Vidom on

In [2]:
def create_vis_plot(_xlabel, _ylabel, _title, _legend):
    return viz.line(
        X=torch.zeros((1,)).cpu(),
        Y=torch.zeros((1, 1)).cpu(),
        opts=dict(
            xlabel=_xlabel,
            ylabel=_ylabel,
            title=_title,
            legend=_legend
        )
    )

In [3]:
def update_vis_plot(iteration, loss, window1, window2, update_type,
                    epoch_size=1):
    viz.line(
        X=torch.ones((1, 1)).cpu() * iteration,
        Y=torch.Tensor([loss]).unsqueeze(0).cpu() / epoch_size,
        win=window1,
        update=update_type
    )

In [4]:
import visdom
viz = visdom.Visdom()

vis_title = 'yolov1_martin_visioNoob.PyTorch on ' + 'VOC'
vis_legend = ['Train Loss']
iter_plot = create_vis_plot('Iteration', 'Loss', vis_title, vis_legend)
epoch_plot = create_vis_plot('Epoch', 'Loss', vis_title, vis_legend)

'>' not supported between instances of 'float' and 'NoneType'
'>' not supported between instances of 'float' and 'NoneType'
'>' not supported between instances of 'float' and 'NoneType'
Visdom python client failed to establish socket to get messages from the server. This feature is optional and can be disabled by initializing Visdom with `use_incoming_socket=False`, which will prevent waiting for this request to timeout.


# 1. Device configuration

In [5]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

num_epochs = 16000
num_classes = 21
batch_size = 1
learning_rate = 1e-4

dropout_prop = 0.5

# 2. Load VOC Pascal'12 Dataset

In [6]:
DATASET_PATH_MARTIN = "/media/keti-ai/AI_HARD3/DataSets/VOC_Pascal/VOC/VOCdevkit/VOC2012"
DATASET_PATH_JAEWON = "H:\VOC\VOC12\VOCdevkit_2\VOC2012"
train_dataset = VOC(root = DATASET_PATH_JAEWON,
                    transform=transforms.ToTensor(), cls_option = True, selective_cls="person")

'>' not supported between instances of 'float' and 'NoneType'


'>' not supported between instances of 'float' and 'NoneType'


VOC Parsing:   |████████████████████████████████████████| 100.0% (17125/17125)  Complete




'>' not supported between instances of 'float' and 'NoneType'


YOLO Generating:|███████████████████████████████████████-| 100.0% (9582/9583)  Complete

YOLO Generating:|████████████████████████████████████████| 100.0% (9583/9583)  Complete



In [7]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size = batch_size,
                                           shuffle = True,
                                           collate_fn=detection_collate)

# 3. Load YOLOv1

In [8]:
net = yolov1.YOLOv1()
# visualize_weights_distribution(net)

model = torch.nn.DataParallel(net, device_ids=[0]).cuda()

summary(model, (3, 448,448))

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95)

'>' not supported between instances of 'float' and 'NoneType'
'>' not supported between instances of 'float' and 'NoneType'


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           9,472
       BatchNorm2d-2         [-1, 64, 224, 224]             128
         LeakyReLU-3         [-1, 64, 224, 224]               0
         MaxPool2d-4         [-1, 64, 112, 112]               0
            Conv2d-5        [-1, 192, 112, 112]         110,784
       BatchNorm2d-6        [-1, 192, 112, 112]             384
         LeakyReLU-7        [-1, 192, 112, 112]               0
         MaxPool2d-8          [-1, 192, 56, 56]               0
            Conv2d-9          [-1, 128, 56, 56]          24,704
      BatchNorm2d-10          [-1, 128, 56, 56]             256
        LeakyReLU-11          [-1, 128, 56, 56]               0
           Conv2d-12          [-1, 256, 56, 56]         295,168
      BatchNorm2d-13          [-1, 256, 56, 56]             512
        LeakyReLU-14          [-1, 256,

# 4.Train the model

In [10]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):

    if (epoch == 200) or (epoch == 400) or (epoch == 600) or (epoch == 20000) or (epoch == 30000):
        scheduler.step()

    for i, (images, labels) in enumerate(train_loader):

    
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)

        # Calc Loss
        loss, \
        obj_coord1_loss, \
        obj_size1_loss, \
        obj_coord2_loss, \
        obj_size2_loss, \
        obj_class_loss, \
        noobj_class_loss, \
        objness1_loss, \
        objness2_loss = yolov1.detection_loss(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 10 == 0:

            print('Epoch:[{}/{}], Step:[{}/{}], learning rate:{}\ttotal_loss\t{:.4f}\tcoord1\t{}\tsize1\t{}\tcoord2\t{}\tsize2\t{}\tclass\t{}\tnoobj_clss\t{}\tobjness1\t{}\tobjness2\t{}'
                  .format(epoch + 1,
                          num_epochs,
                          i + 1,
                          total_step,
                          [param_group['lr'] for param_group in optimizer.param_groups],
                          loss.item(),
                          obj_coord1_loss,
                          obj_size1_loss,
                          obj_coord2_loss,
                          obj_size2_loss,
                          obj_class_loss,
                          noobj_class_loss,
                          objness1_loss,
                          objness2_loss
                          ))
            

            yolov1.update_vis_plot(viz, (epoch+1)*batch_size +(i + 1), loss.item(), iter_plot, None, 'append')
            yolov1.update_vis_plot(viz, (epoch + 1) * batch_size + (i + 1), obj_coord1_loss, coord1_plot, None, 'append')
            yolov1.update_vis_plot(viz, (epoch + 1) * batch_size + (i + 1), obj_size1_loss, size1_plot, None, 'append')
            yolov1.update_vis_plot(viz, (epoch + 1) * batch_size + (i + 1), obj_coord2_loss, coord2_plot, None, 'append')
            yolov1.update_vis_plot(viz, (epoch + 1) * batch_size + (i + 1), obj_size2_loss, size2_plot, None, 'append')
            yolov1.update_vis_plot(viz, (epoch + 1) * batch_size + (i + 1), obj_class_loss, obj_cls_plot, None, 'append')
            yolov1.update_vis_plot(viz, (epoch + 1) * batch_size + (i + 1), noobj_class_loss, noobj_cls_plot, None, 'append')
            yolov1.update_vis_plot(viz, (epoch + 1) * batch_size + (i + 1), objness1_loss, objectness1_plot, None, 'append')
            yolov1.update_vis_plot(viz, (epoch + 1) * batch_size + (i + 1), objness2_loss, objectness2_plot, None, 'append')


            

    if (epoch % 300) == 0:
        yolov1.save_checkpoint({
            'epoch': epoch + 1,
            'arch': "YOLOv1",
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict(),
        }, False, filename='checkpoint_{}.pth.tar'.format(epoch))


RuntimeError: cuda runtime error (2) : out of memory at c:\users\administrator\downloads\new-builder\win-wheel\pytorch\aten\src\thc\generic/THCStorage.cu:58

'>' not supported between instances of 'float' and 'NoneType'
'>' not supported between instances of 'float' and 'NoneType'
'>' not supported between instances of 'float' and 'NoneType'
'>' not supported between instances of 'float' and 'NoneType'
'>' not supported between instances of 'float' and 'NoneType'
'>' not supported between instances of 'float' and 'NoneType'
'>' not supported between instances of 'float' and 'NoneType'
'>' not supported between instances of 'float' and 'NoneType'
